In [1]:
import pandas as pd
import numpy as np
from util.process import data_process2
from util.process import word_ngrams
import gensim
from util.query2vec import trans_vec
from sklearn import metrics

In [2]:
#数据读取
test_data=pd.read_csv('./data/test_data.csv')

## 读取模型(读取多个中的一个)

In [3]:
model = gensim.models.Word2Vec.load('./model/word2vec_baike')
embedding_dim=256

In [6]:
model= gensim.models.KeyedVectors.load_word2vec_format('./model/cc.zh.300.vec')
embedding_dim=300

In [28]:
#glove
model=gensim.models.KeyedVectors.load_word2vec_format('./model/w2v_glove.txt') 
embedding_dim=300

In [ ]:
import time

In [ ]:
start=time.time()
trans_vec(data_process('哪里是中国近代工业文明的主要发祥地'),model,embedding_dim
end=time.time()
print(end-start)

### 验证模型

In [4]:
def sim(x, y):
    if x.any() and y.any():
        res = np.array([[x[i] * y[i], x[i] * x[i], y[i] * y[i]] for i in range(len(x))])
        cos = sum(res[:, 0]) / (np.sqrt(sum(res[:, 1])) * np.sqrt(sum(res[:, 2])))
        return cos      
    else:
        return float(1) if x.any() == y.any() else float(0)
def get_pred(texta_list,textb_list,model,embedding_dim):
    texta_list=list(map(lambda x:trans_vec(data_process2(x),model,embedding_dim=embedding_dim),texta_list))
    textb_list=list(map(lambda x:trans_vec(data_process2(x),model,embedding_dim=embedding_dim),textb_list))
    y_pred=[sim(a,b) for a,b in zip(texta_list,textb_list)]
    y_pred=np.array(y_pred)
    y_pred[np.isnan(y_pred)]=0
    return y_pred

In [ ]:
y_pred=get_pred(list(test_data['testa']),list(test_data['testb']),model,embedding_dim)
y_true=test_data['label']   

In [16]:
metrics.roc_auc_score(y_true,y_pred)

0.9342238155051089

In [17]:
y_pred1=list(map(lambda x:1 if x>0.85 else 0,y_pred))
metrics.accuracy_score(y_true,y_pred1)

0.7634781064037659

In [103]:
save_data=pd.DataFrame()
save_data['y_tue']=y_true
save_data['y_pred']=y_pred
save_data.to_csv('./prob/ft_prob.csv',index=None)